# Travelling Salesman Problem optimizaion using Genetic Algorithm

This is my firt project in Python. Motivate for learning Python is primarily to code AI and ML algorithms. As I have done this project to test myself on Python basics, I have on-purpose avoided use of advanced python packages such numpy or pandas. The program imports only 2 packages - random (to shuffle and generate random numbers) and xlrd (to read excel file).

## Travelling Salesman Problem (TSP)
I chose TSP as a good candidate to solve, as its a simple problem to understand and yet, solving via brute-force, it grows in complexity by n! (n factorial). 
https://en.wikipedia.org/wiki/Big_O_notation
https://en.wikipedia.org/wiki/Travelling_salesman_problem
TSP asks the following question: "Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city and returns to the origin city?

## Genetic Algorithm (GA)
https://en.wikipedia.org/wiki/Genetic_algorithm
Genetic Algorithm is a heuristic algorithm inspired by process of natural selection based on the concept of Darwin's theory of evolution. GA is one of the meta-heuristics that has been frequently utilized to find near-optimum solutions of many combinational problems. In GA, a randomly generated 'population' of possible solutions is iteratively 'mated' to produce successive generations of 'children' population, which converge toward better and better solutions.

https://iccl.inf.tu-dresden.de/w/images/b/b7/GA_for_TSP.pdf

The above is the primary paper I have referenced to develop my GA algorithm code. As the paper notes, TSP is not a straight forward candidate to apply GA, as encoding of a TSP solution as a bit string is not convenient. TSP, as opposed to most problems tackled by GA, is a pure ordering problem. Hence, the original GA algorithm must be modified to handle combinatorial optimization problems like TSP. The paper thoroughly discusses various extensions of GA for 'encoding', 'mating' and 'mutating' solution population. It also gives computational results comparing the superiority of the various extensions.

I have selected the "Order Crossover (OX)" as mating technique along with simple 'Swap' technique for mutation. Short explanation of the 2 techniques is below

Order Crossover (OX):
This allows two cut points to be randomly chosen on the parent chromosomes. In order to create an offspring, the string between the two cut points in the first parent is first copied to the offspring. Then, the remaining positions are filled by considering the sequence of cities in the second parent, starting after the second cut point (when the end of the chromosome is reached, the sequence continues at position 1). The OX tries to prserve the order of cities in parents rathen than absolute position. Example

parent 1: 1 2 | 5 6 4 | 3 8 7

parent 2: 1 4 | 2 3 6 | 5 7 8

offspring

(step 1): - - | 5 6 4 | - - -

(step 2): 2 3 | 5 6 4 | 7 8 1


In step 2, city 5 is first considered to occupy position 6, but it is discarded because it is already included in the offspring. City 7 is the next city to be considered, and it is inserted at position 6. Then, city 8 is inserted at position 7, city 1 is inserted at position 8, city 4 is discarded, city 2 is inserted at position 1, city 3 is inserted at position 2, and city 6 is discarded.

Swap:
Two cities are randomly selected and swapped (i.e. their positions are exchanged). This mutation operator is the closest in philosophy to the original mutation operator, because it only slightly modifies the original tour.

## Format of Distance matrix:
I have considered city 0 as both starting and ending position for a complete tour. The format of tabulating the distance bewteen each of the cities in excel is as given below for a sample of 8 cities

0	1	2	3	4	5	6	7	8
0	61	88	15	33	62	73	81	56
51	0	52	70	72	19	48	19	5
78	42	0	25	2	7	31	51	36
5	60	15	0	59	76	80	65	44
23	62	12	49	0	90	90	31	33
52	9	17	66	80	0	52	68	63
63	38	21	70	80	42	0	7	94
71	9	41	55	21	58	17	0	80
46	15	26	34	23	53	84	70	0

The first row represents the number of the city. Its always [0 1 2 3 4] etc. There are no columns representing the cities, as it can be readily understood from the table. 2nd row is for city 0, 3rd row for city 1 and so on. I have eliminated the city column to simplify reading the excel. Once the table is read, the first row, representing the city is deleted, giving a table (2D list), where city 0 is at index 0, city 1 at index 1 and so on. This becomes extremely convenient to code, as index can directly be referenced in lieu of intended city.

The sample distances are randomly generated and manually modified few cells (for 8 city sample in sheet1) to make the table asymmetric, adding bit of complexity. All distances are between 1 to 100. In case actual distances are beyond 1 to 100 range, I suggest normalizing distances between 1 to 100. Otherwise it may affect the Fit_Inverse calculation in the code. Sheet2 has larger sample with 50 cities.

## Description of variables and functions

Dist_Grid : Distance Grid. 2D list containing the distances between all 2 city pairs. Index of the list represents the city number. For example Dist_Grid[0][2] gives distance from city 0 to city 2. Dist_Grid[5][3] distance from city 5 to city 3
Get_Dist_Data: Function to get Dist_Grid 2D list. Reads excel containing distances between cities encoded in the above mentioned format. Deletes the header row and returns Dist_Grid
Popln_size : Size of the population
Iterations : Number of children generations to run the algorithm
Mutate_Freq : Mutation frequency. Once a child is created, it is mutated with this probability
Length : Number of cities excluding city 0
Popln : Parent population of any generation
Children : Children population of any generation, after mating of parents
Popln_fit : Total length of the route encoded in each parent. Actually this represents inverse of fitness. Higher the route length lower is the fitness
Fit_Inverse : This list represents true fitness of each parent. The inverse of the route lenght is raised to a power in order to give higher weightage to better solutions. Without raising to power the algorithm was not converging. Also multiplied by 10,000 to avoid values less than 1 before squaring
Fit_Power : The power to which the Fit_Inverse is raised. Higher the power, more weightage to fitter members
Best_Member : Best route discovered across any generation
Best_Member_Fit : Shortest route length discovered across any generation. Basically route lenght of Best_Member
gen_popln : Function that generates initial random set of parent population
fit_func : Function that calculates route length of each parent
mate : Function that mates 2 parents as per Order Crossover technique and returns the child
mutate: Function that mutates child as per swap technique

In [7]:
import random
import xlrd

Dist_Grid = Get_Dist_Data()

Popln_size = 20 ; Iterations = 2000 ; Mutate_Freq = 0.01 ; Fit_Power = 5

Length = len (Dist_Grid) -1 ; Popln = [] ; Children = [] ; Popln_fit = [] ; Fit_Inverse = []

Best_Member = [] ; Best_Member_Fit = 0.0

Popln = gen_popln (Length, Popln_size) # Generate 1st set of random parent population
Popln_fit = fit_func (Popln) # Calculate route lengths for each parent of the population
Best_Member_Fit = min (Popln_fit)
Best_Member_Fit_Initial = min (Popln_fit)
Best_Member = Popln [Popln_fit.index(Best_Member_Fit)]

for Gen in range (Iterations):
    Children = [] ; Fit_Inverse = [] # Reset Children and Fit_Inverse lists for each Generation
    for k in Popln_fit:
            Fit_Inverse.append (int(1/k*10000)**Fit_Power) #Calculate inverse of route length and raise it to power. 10,000 multiplier to avoid inverse value below 1 
    for j in range (Popln_size):
        Children.append (mate (random.choices(Popln, weights = Fit_Inverse, k=1)[0], random.choices(Popln, weights = Fit_Inverse, k=1)[0]))
        #Above line picks two parents from population to mate. Higher weightage for members with better fitness (shorter route lenght). 
        #[0] is used since random.choices returns 2D list with one member (as k-1). [0] converts it to single member list
        if Mutate_Freq > random.random():
            Children[-1] = mutate (Children[-1]) # Mutates the last member in the Children population
    Popln = Children # Assign Children as parents for next generation
    Popln_fit = fit_func (Popln) # Calculate fitness of Child population (new parents)
    if min (Popln_fit) < Best_Member_Fit: # Check if any member is better than previous generation. If yes, then store as best member and fitness
        Best_Member_Fit = min (Popln_fit)
        Best_Member = Popln [Popln_fit.index(Best_Member_Fit)]
        
print ("Best route length found: ", Best_Member_Fit)
print ("Best route: ", Best_Member)
print ("Best route initially found: ", Best_Member_Fit_Initial)
print ("Improvement over initial route: ", (Best_Member_Fit_Initial - Best_Member_Fit) / Best_Member_Fit_Initial * 100, "%")

Best route length found:  761.0
Best route:  [3, 44, 28, 31, 9, 38, 42, 48, 24, 11, 46, 8, 29, 30, 43, 14, 23, 7, 37, 34, 50, 35, 1, 4, 32, 19, 27, 10, 17, 36, 13, 6, 15, 5, 21, 12, 26, 41, 47, 2, 39, 25, 22, 49, 40, 16, 33, 45, 18, 20]
Best route initially found:  2037.0
Improvement over initial route:  62.64113892979872 %


In [2]:
def Get_Dist_Data ():
    path = 'TSP_City Dist Data_v2.xlsx'
    WB = xlrd.open_workbook(path)
    Sheet = WB.sheet_by_index(1)
    Dist_Grid = []
    for rownum in range (Sheet.nrows):
        Dist_Grid += [Sheet.row_values(rownum)]
    Dist_Grid.remove(Dist_Grid[0]) # Deletes the header row
    return Dist_Grid

In [3]:
def mate (Parent1, Parent2):
    Crssover1 = random.randint(0,Length-1) # First random crossover point
    Crssover2 = random.randint(Crssover1+1,Length) # Second random crossover point
    Child_temp1 = Parent1[Crssover1:Crssover2] # Selects string between the crossover points from Parent 1
    Child_temp2 = Parent2[Crssover2:Length]+Parent2[0:Crssover2] # Selects string outside of crossover points from Parent 2. Starts after crossover 2 till end. Then rollsover to start till 1st corssover 
    Child_temp3 = Parent2[Crssover2:Length]+Parent2[0:Crssover2] # Same as Child_temp2
    for position in Child_temp2:         # This loop deletes all duplicate cities from Parent 2 which are already picked from Parent 1 (between the crossover points)
        if position in Child_temp1:
            Child_temp3.remove (position) 
    Child = Child_temp1 + Child_temp3[0: Length-Crssover2] # adds string from Parent 2 after crossover point 2
    Child = Child_temp3[Length-Crssover2:] + Child # adds string from Parent 2 before crossover point 1
    return Child

In [4]:
def mutate (Child):
    mutate_position1 = random.randint (0, Length-1)
    mutate_position2 = random.randint (0, Length-1)
    Child[mutate_position1], Child[mutate_position2] = Child[mutate_position2], Child[mutate_position1] # Swaps 2 random cities
    return Child

In [5]:
def gen_popln (Length, Popln_size):
    for num in range (Popln_size):
        Member = list (range(1,Length+1)) # Generates sequence 1,2,3,.. till number of cities
        random.shuffle (Member) # Shuffles the order of cities, creating a random route
        Popln.append(Member)
    return Popln

In [6]:
def fit_func (Popln):
    Popln_fit = []
    for num in Popln: # Loops through each member of the population as num
        fit = Dist_Grid[0][num[0]] # Adds distance from city 0 to the first city in the current member (which is num)
        for i in range (len(num)-1): # Loop adds the distance between cities as per order in the member
            fit += Dist_Grid[ num[i] ][ num[i+1] ]
        fit += Dist_Grid[num[len(num)-1]][0] # Adds distance from last city back to city 0
        Popln_fit.append(fit) # Appends the total route distance to the respective position in Popln_fit list
    return Popln_fit    

## Results

I tried various combinations of Population size, no. of iterations, Mutation probability and Fitness power. 

8 cities sample data:
Got good results with 200 iterations and Fitness power of 2. Algorithm was able to find best route within 10% of absolute best in 5 out of 10 cases. Below are results

Popln_size = 20 ; Iterations = 200 ; Mutate_Freq = 0.01 ; Fit_Power = 2

Run	Best Route length	Initial Best route length	% better than initial	% gap from absolute best route (156)
1	204	                204	                        0%	                    31%
2	194	                307	                        37%	                    24%
3	162	                336	                        52%	                    4%
4	186	                287	                        35%	                    19%
5	193	                321	                        40%	                    24%
6	156	                300	                        48%	                    0%
7	162	                210	                        23%	                    4%
8	162	                270	                        40%	                    4%
9	156	                262	                        40%	                    0%
10	186	                298	                        38%	                    19%


50 cities sample data:
Got good results with 2000 iterations and Fit_Power of 5. The absolute best route of 708 is the best one the algorithm found across severals runs with varying parameters. Have high degree of confidence that this is very close absolute best (The average distance travelled between cities is less than 14 for this solution). Algorithm was able to find best route within 10% of absolute best in 3 out of 20 cases. More importantly, the solution was able to reduce the route length by more than half in all cases except one and in several cases reduced route length by more than a third. Below are results

Popln_size = 20 ; Iterations = 2000 ; Mutate_Freq = 0.01 ; Fit_Power = 5

Run	Best Route length	Initial Best route length	% better than initial	% gap from absolute best route (708)
1	963	                2261	                    57%	                    36%
2	775	                2438	                    68%	                    9%
3	1033	            2305	                    55%	                    46%
4	776	                2259	                    66%	                    10%
5	1124	            1957	                    43%                 	59%
6	1039	            2191	                    53%	                    47%
7	955	                2137	                    55%	                    35%
8	708	                2273	                    69%	                    0%
9	980	                2200	                    55%	                    38%
10	857	                2279	                    62%	                    21%
11	839	                2203	                    62%	                    19%
12	849	                2098	                    60%	                    20%
13	960	                1888	                    49%	                    36%
14	960	                2173	                    56%	                    36%
15	950	                2110	                    55%	                    34%
16	830	                2203	                    62%	                    17%
17	1054	            2127	                    50%	                    49%
18	829	                2402	                    65%	                    17%
19	900	                2349	                    62%	                    27%
20	822	                2363	                    65%	                    16%